### **Project 01: Data Preprocessing and Classification**

In [1]:
from google.colab import drive
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

Drive not mounted, so nothing to flush and unmount.
All changes made in this colab session should now be visible in Drive.


The following code cell installs additional dependencies required to run the Jupyter Notebooks used in this class.

In [2]:
# Add additional Python packages that we will be using in class
!apt install graphviz build-essential checkinstall imagemagick
# Base Python packages to run example Jupyter Notebooks
!pip install watermark pyprind mlxtend
# Python packages to visualize Decision Tree Classifiers
!pip install pydotplus graphviz pyparsing
# Python packages for Natrual Language Processing
!pip install nltk
# Python packages for Flask-based web applications
!pip install flask wtforms
# Python packages for TensorFlowdf.select_dtypes(include=['int64','float64']).columns
!pip install tensorflow tensorflow-datasets

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
graphviz is already the newest version (2.40.1-2).
The following additional packages will be installed:
  file fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagic-mgc libmagic1 libmagickcore-6.q16-3
  libmagickcore-6.q16-3-extra libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7
  netpbm poppler-data
Suggested packages:
  gettext fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr
  | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray
  radiance sane-utils texlive-base-bin transfig ufraw-batch inkscape
  libjxr-tools libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ip

The following code cell defines a variable equal to the location inside your Google Drive where you copied the ch03 folder

In [3]:
##### TODO CHANGE THIS TO THE PATH IN GOOGLE DRIVE WHERE YOU COPIED THE ch03 FOLDER #####
google_drive_root='/Colab Notebooks/'

In [4]:
google_drive_mount_location = '/content/drive'
google_file_prefix=google_drive_mount_location + '/My Drive/' + google_drive_root + '/'

The following code cell mounts your Google Drive into the runtime of the workbook, so that you can access files.

In [5]:
# Read more here: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=D78AM1fFt2ty
from google.colab import drive
drive.mount(google_drive_mount_location)

Mounted at /content/drive


In [6]:
%load_ext watermark
%watermark -a "Sebastian Raschka" -u -d -p numpy,pandas,matplotlib,sklearn

Author: Sebastian Raschka

Last updated: 2021-10-20

numpy     : 1.19.5
pandas    : 1.1.5
matplotlib: 3.2.2
sklearn   : 0.0



In [7]:
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns

## **The goal of this project is to predict gender of student based on the data.**

## 1. Data Collection

https://data.colorado.gov/Higher-Education/degreesAwarded/m279-sgtm

Data pulled from Colorado Information Marketplace

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/degreesAwarded.csv', header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,9,10,15,17,18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,year,institutionName,institutionLevelId,institutionLevel,institutionType,taxType,coBased,division,ageDesc,ageMin,ageMax,gender,ethnicity,studentLevel,degreeLevel,residencyId,residency,cip2,cip,programName,recordCount
1,2001,Aims Community College,2,College,Public,Not-for-Profit,Yes,DHE,Under 17,NaN,17,Female,"White, non-Hispanic",Undergraduate,Certificate,1,In-State,51,51.1614,NaN,NaN
2,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,Unknown,Undergraduate,Bachelor,1,In-State,31,31.0301,"Parks, Recreation and Leisure Facilities Manag...",NaN
3,2001,Aims Community College,2,College,Public,Not-for-Profit,Yes,DHE,17-20,17,20,Female,Hispanic,Undergraduate,Certificate,1,In-State,20,20.0201,NaN,NaN
4,2001,Aims Community College,2,College,Public,Not-for-Profit,Yes,DHE,17-20,17,20,Female,Hispanic,Undergraduate,Certificate,1,In-State,51,51.1614,NaN,NaN


## **2. Data Preprocessing**

Looks like the column headers are the indices while the actual headers are the first row. Let's fix the column names.

In [10]:
df.columns = df.iloc[0].values
df = df.iloc[1:,:]

In [11]:
df.head()

,year,institutionName,institutionLevelId,institutionLevel,institutionType,taxType,coBased,division,ageDesc,ageMin,ageMax,gender,ethnicity,studentLevel,degreeLevel,residencyId,residency,cip2,cip,programName,recordCount
1,2001,Aims Community College,2,College,Public,Not-for-Profit,Yes,DHE,Under 17,NaN,17,Female,"White, non-Hispanic",Undergraduate,Certificate,1,In-State,51,51.1614,NaN,NaN
2,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,Unknown,Undergraduate,Bachelor,1,In-State,31,31.0301,"Parks, Recreation and Leisure Facilities Manag...",NaN
3,2001,Aims Community College,2,College,Public,Not-for-Profit,Yes,DHE,17-20,17,20,Female,Hispanic,Undergraduate,Certificate,1,In-State,20,20.0201,NaN,NaN
4,2001,Aims Community College,2,College,Public,Not-for-Profit,Yes,DHE,17-20,17,20,Female,Hispanic,Undergraduate,Certificate,1,In-State,51,51.1614,NaN,NaN
5,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,Unknown,Undergraduate,Bachelor,1,In-State,31,31.0301,"Parks, Recreation and Leisure Facilities Manag...",NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221941 entries, 1 to 221941
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   year                221941 non-null  object
 1   institutionName     213650 non-null  object
 2   institutionLevelId  213650 non-null  object
 3   institutionLevel    213650 non-null  object
 4   institutionType     213650 non-null  object
 5   taxType             213650 non-null  object
 6   coBased             213650 non-null  object
 7   division            213650 non-null  object
 8   ageDesc             213650 non-null  object
 9   ageMin              212800 non-null  object
 10  ageMax              178649 non-null  object
 11  gender              221941 non-null  object
 12  ethnicity           221941 non-null  object
 13  studentLevel        221941 non-null  object
 14  degreeLevel         221941 non-null  object
 15  residencyId         221941 non-null  object
 16  re

These are all in object types. At least the age should be numerical.

In [13]:
df.isnull().sum()

year                       0
institutionName         8291
institutionLevelId      8291
institutionLevel        8291
institutionType         8291
taxType                 8291
coBased                 8291
division                8291
ageDesc                 8291
ageMin                  9141
ageMax                 43292
gender                     0
ethnicity                  0
studentLevel               0
degreeLevel                0
residencyId                0
residency                  0
cip2                       4
cip                        0
programName            24159
recordCount           207492
dtype: int64

That's quite a lot of missing values, but with the sheer size of the dataset, we can probably still make do with even if we blindly removed all rows with empty values.

In [14]:
df['ageMin'].unique()

array([nan, '17', '21', '25', '40', 25.0, 21.0, 40.0, 17.0], dtype=object)

In [15]:
df['ageMax'].unique()

array(['17', nan, '20', '24', '40', 40.0, 24.0, 20.0, 17.0], dtype=object)

The 'ageMin' and 'ageMax' are object types. Let's clean them up and convert them to numericals. 

'ageMax' has a lot more missing values than 'ageMin'.

In [16]:
df['ageMin'] = pd.to_numeric(df['ageMin'])

In [17]:
# Let's see if it worked
df['ageMin'].unique()

array([nan, 17., 21., 25., 40.])

In [18]:
df['ageMax'] = pd.to_numeric(df['ageMax'])

In [19]:
df['ageMax'].unique()

array([17., nan, 20., 24., 40.])

In [20]:
df['ageMin'].unique()

array([nan, 17., 21., 25., 40.])

In [21]:
df['ageMedian'] = (df['ageMin'] + df['ageMax'])/2

Instead of using two age metrics, let's simplify ageMin and ageMax a new feature called 'ageMedian' that takes the Max and the Min and outputs the median value of the two. 

We could also impute the nan values here as well by looking at the ages of people based on the 'degreeLevel' and 'gender' impute their ages based on the averages of those. 

degreeLevel - data on whether one is an undergrad, grad or phd student is a good predictor of age. Very rare to see extremes where phd students are very young or undergraduates very old.

gender - just to see if there's a difference.

**Note: in this case, the median is the same as the average.**

In [22]:
df['ageMedian'].unique()

array([ nan, 18.5, 22.5, 32.5])

In [23]:
df.groupby(['degreeLevel','gender'])['ageMedian'].agg(['mean','count'])

mean  count
degreeLevel   gender                   
Associate     Female   26.234812  17942
              Male     25.994725  15544
              Unknown  26.983871    186
Bachelor      Female   25.723257  38337
              Male     26.357123  36367
              Unknown  29.868421     57
Certificate   Female   26.363056  18241
              Male     25.978891  17125
              Unknown  25.712308    325
Doctoral      Female   32.460568   2536
              Male     32.486550   2974
              Unknown  32.500000      1
Masters       Female   29.668543  14038
              Male     30.063651  12600
              Unknown  32.500000     46
OtherGraduate Male     32.500000      1
Professional  Female   30.432285    827
              Male     30.938003    621
Specialist    Female   31.880531    113
              Male     32.074468     47
              Unknown  32.500000      1

Looks like there's not much of a difference.

Interesting to note:

Slightly more females - Associate, Bachelor, Certificate (roughly the same), Masters, Professional, and Specialist.

More males - Doctoral

Let's fill in the NaN age values with the corresponding professions and gender.

In [24]:
df.isnull().sum()

year                       0
institutionName         8291
institutionLevelId      8291
institutionLevel        8291
institutionType         8291
taxType                 8291
coBased                 8291
division                8291
ageDesc                 8291
ageMin                  9141
ageMax                 43292
gender                     0
ethnicity                  0
studentLevel               0
degreeLevel                0
residencyId                0
residency                  0
cip2                       4
cip                        0
programName            24159
recordCount           207492
ageMedian              44012
dtype: int64

In [25]:
df['institutionLevelId'].unique()

array(['2', nan, '4', 2, 4], dtype=object)

In [26]:
df['institutionName'].unique()

array(['Aims Community College', nan,
       'University of Colorado Colorado Springs',
       'University of Colorado Boulder', 'Arapahoe Community College',
       'Pikes Peak Community College', 'Colorado School of Mines',
       'Colorado State University', 'Red Rocks Community College',
       'Colorado State University - Pueblo', 'Colorado Mountain College',
       'Colorado Mesa University', 'CSU-Global Campus',
       'University of Colorado Denver', 'Community College of Aurora',
       'Community College of Denver', 'Front Range Community College',
       'Lamar Community College', 'Otero Junior College',
       'Morgan Community College', 'Northeastern Junior College',
       'Pueblo Community College', 'Trinidad State Junior College',
       'Adams State University',
       'Metropolitan State University of Denver', 'Fort Lewis College',
       'University of Northern Colorado',
       'Western State Colorado University',
       'Colorado Northwestern Community College'], d

In [27]:
df['institutionType'].unique()

array(['Public', nan], dtype=object)

In [28]:
df = df.dropna(subset=['institutionType'])

In [29]:
df = df.dropna(subset=['programName'])

In [30]:
df.isnull().sum()

year                       0
institutionName            0
institutionLevelId         0
institutionLevel           0
institutionType            0
taxType                    0
coBased                    0
division                   0
ageDesc                    0
ageMin                   772
ageMax                 31081
gender                     0
ethnicity                  0
studentLevel               0
degreeLevel                0
residencyId                0
residency                  0
cip2                       0
cip                        0
programName                0
recordCount           177494
ageMedian              31736
dtype: int64

In [31]:
grp = df.groupby(['degreeLevel','gender'])['ageMedian'].mean()

In [32]:
grp = df.groupby(['degreeLevel','gender'])['ageMedian'].mean().reset_index()[['degreeLevel','gender','ageMedian']]

In [33]:
def fill_age(x):
    return grp[(grp.degreeLevel==x.degreeLevel)&(grp.gender==x.gender)]['ageMedian'].values[0]

In [34]:
df['ageMedian'] = (df.apply(lambda x: fill_age(x) if np.isnan(x['ageMedian']) else x['ageMedian'], axis=1) for i in df)

In [35]:
df.isnull().sum()

year                       0
institutionName            0
institutionLevelId         0
institutionLevel           0
institutionType            0
taxType                    0
coBased                    0
division                   0
ageDesc                    0
ageMin                   772
ageMax                 31081
gender                     0
ethnicity                  0
studentLevel               0
degreeLevel                0
residencyId                0
residency                  0
cip2                       0
cip                        0
programName                0
recordCount           177494
ageMedian                  0
dtype: int64

Sweet. Looks like we got rid of all the missing values for ageMedian. Let's drop 'ageMin' and 'ageMax'

In [36]:
df = df.drop(['ageMin','ageMax'],axis=1)

In [37]:
df['institutionLevelId'].unique()

array(['4', '2', 2, 4], dtype=object)

It should be either 2 or 4. Let's consolidate these.

In [38]:
df['institutionLevelId'] = pd.to_numeric(df['institutionLevelId'])

In [39]:
df['institutionLevelId'].unique()

array([4, 2])

In [40]:
df['institutionLevelId'] = df['institutionLevelId'].astype(object)

In [41]:
df['coBased'].unique()

array(['Yes'], dtype=object)

It only has 1 value. Put this on the list of features to drop.

In [42]:
df['institutionLevel'].unique()

array(['University', 'College'], dtype=object)

Only two classes. degreeLevel has more granularity, so this is unnecessary. Drop.

In [43]:
df['institutionType'].unique()

array(['Public'], dtype=object)

Single value. Drop.

In [44]:
df['taxType'].unique()

array(['Not-for-Profit'], dtype=object)

Single value. Drop.

In [45]:
df['division'].unique()

array(['DHE'], dtype=object)

In [46]:
df['ageDesc'].unique()

array(['21-24', 'Over 40', '25-40', '17-20', 'Under 17', 'Unknown'],
      dtype=object)

In [47]:
df['ethnicity'].unique()

array(['White, non-Hispanic',
       'More than one race/ethnicity (non-hispanic)', 'Unknown Ethnicity',
       'Unknown', 'Black or African American, non-Hispanic',
       'Non-Resident Alien', 'Asian', 'Hispanic',
       'Native American or Alaskan Native', 'Asian or Pacific Islander',
       'Hawaiian or Pacific Islander', 'Black or African American'],
      dtype=object)

Might consider mapping some of these ethnicities to Race instead to simplify things. Black, Asian, White.

In [48]:
df['studentLevel'].unique()

array(['Undergraduate', 'Graduate'], dtype=object)

In [49]:
df['residencyId'].unique()

array(['2', '1', '3', 1, 2, 3], dtype=object)

In [50]:
df['residencyId'] = pd.to_numeric(df['residencyId'])
df['residencyId'] = df['residencyId'].astype(object)

In [51]:
df['residency'].unique()

array(['Out-of-State', 'In-State', 'Unclassified/Unspecified'],
      dtype=object)

Not sure what to do with this feature yet.

In [54]:
df['programName'].unique()

array(['Electrical and Electronics Engineering',
       'Business Administration and Management, General',
       'Economics, General', 'History, General',
       'Kinesiology and Exercise Science',
       'Medical Office Management/Administration',
       'Mechanical Engineering', 'Civil Engineering, General',
       'Criminal Justice/Law Enforcement Administration',
       'Psychology, General', 'Legal Assistant/Paralegal',
       'Fire Science/Fire-fighting',
       'Biology/Biological Sciences, General', 'Art/Art Studies, General',
       'Political Science and Government, General',
       'Library and Information Science', 'Reading Teacher Education',
       'Public Health, General', 'Curriculum and Instruction',
       'Child Development', 'Music Performance, General',
       'Welding Technology/Welder',
       'Liberal Arts and Sciences, General Studies and Humanities, Other',
       'Cooking and Related Culinary Arts, General',
       'Computer/Information Technology Services A

Might want to feature engineer these into more broader group such as:
- STEM
- Humanities
- Life Sciences
- Art
- Administrative
- Religious Studies
- Teaching
- Linguistics
- Literature
- Social Sciences
- IT
- History
- Trade


In [ ]:
def fill_age(x):
    return grp[(grp.degreeLevel==x.degreeLevel)&(grp.gender==x.gender)]['ageMedian'].values[0]

In [ ]:
df['ageMedian'] = (df.apply(lambda x: fill_age(x) if np.isnan(x['ageMedian']) else x['ageMedian'], axis=1) for i in df)

In [84]:
df['programGrp'] = ''

In [99]:
df['programName'].str.contains('Engineering')

19         True
199        True
279        True
422       False
512       False
          ...  
221936    False
221937    False
221938    False
221939    False
221941    False
Name: programName, Length: 190198, dtype: bool

In [100]:
def fill_stem(df):
  for i in df:
    if df['programName'].str.contains('Engineering') | df['programName'].str.contains('Computer Science') | df['programName'].str.contains('Mathematics'):
      df['programGrp'].loc[i] = 'STEM'

In [101]:
fill_stem(df)

ValueError: ignored

In [72]:
def fill_stem(df):
  return prog_grp[(prog_grp.programName==x.programName)]

In [ ]:
df['programGrp'] = (df.apply(lambda x: fill_stem(x) if (x['programName'].str.contains('Engineering') | df['programName'].str.contains('Computer Science') | df['programName'].str.contains('Mathematics')) else x['programGrp'], axis=1) for i in df)

In [77]:
prog_grp = pd.Series(df['programName'])

In [78]:
prog_grp

19                   Electrical and Electronics Engineering
199                  Electrical and Electronics Engineering
279                  Electrical and Electronics Engineering
422         Business Administration and Management, General
512                                      Economics, General
                                ...                        
221936     Business/Office Automation/Technology/Data Entry
221937                     Kinesiology and Exercise Science
221938    Administrative Assistant and Secretarial Scien...
221939                     Kinesiology and Exercise Science
221941          Sport and Fitness Administration/Management
Name: programName, Length: 190198, dtype: object

In [74]:
df.head()

,year,institutionName,institutionLevelId,institutionLevel,institutionType,taxType,coBased,division,ageDesc,gender,ethnicity,studentLevel,degreeLevel,residencyId,residency,cip2,cip,programName,recordCount,ageMedian
19,2016,University of Colorado Colorado Springs,4,University,Public,Not-for-Profit,Yes,DHE,21-24,Male,"White, non-Hispanic",Undergraduate,Bachelor,2,Out-of-State,14,14.1001,Electrical and Electronics Engineering,NaN,<generator object <genexpr> at 0x7fdd31d6e4d0>
199,2016,University of Colorado Colorado Springs,4,University,Public,Not-for-Profit,Yes,DHE,21-24,Male,More than one race/ethnicity (non-hispanic),Undergraduate,Bachelor,1,In-State,14,14.1001,Electrical and Electronics Engineering,NaN,<generator object <genexpr> at 0x7fdd31d6e4d0>
279,2016,Colorado School of Mines,4,University,Public,Not-for-Profit,Yes,DHE,21-24,Male,"White, non-Hispanic",Undergraduate,Bachelor,1,In-State,14,14.1001,Electrical and Electronics Engineering,18,<generator object <genexpr> at 0x7fdd31d6e4d0>
422,2013,Red Rocks Community College,2,College,Public,Not-for-Profit,Yes,DHE,Over 40,Male,"White, non-Hispanic",Undergraduate,Certificate,1,In-State,52,52.0201,"Business Administration and Management, General",NaN,<generator object <genexpr> at 0x7fdd31d6e4d0>
512,2001,Colorado State University,4,University,Public,Not-for-Profit,Yes,DHE,25-40,Female,Unknown Ethnicity,Graduate,Masters,1,In-State,45,45.0601,"Economics, General",NaN,<generator object <genexpr> at 0x7fdd31d6e4d0>


In [73]:
fill_stem(df)

ValueError: ignored

In [ ]:
program_map = {'STEM':['Electrical and Electronics Engineering',' '],
               'Humanities':[''],
               'Life Sciences':[''],
               'Art':[''],
               'Administrative':['Business Administration and Management',''],
               'Religious Studies':[''],
               'Teaching':[''],
               'Linguistics':[''],
               'Literature':[''],
               'Social Sciences':['Economics, General',''],
               'IT':[''],
               'History':[''],
               'Trade':[''],
               }

In [ ]:
df = df.drop(['cip', 'cip2', 'residency','studentLevel', 'ageDesc','division','taxType','institutionType','institutionLevel','coBased','recordCount'],axis=1)

In [ ]:
df['year'].unique()

In [ ]:
df.isnull().sum()

# **Data Visualization**

We've cleaned up the data. Now on to data visualization.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,4),dpi=150)
sns.countplot(x='institutionName',data=df, hue= 'gender')
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(6,2),dpi=150)
sns.countplot(x='ethnicity',data=df, hue= 'gender')
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(10,4),dpi=150)
sns.countplot(x='degreeLevel',data=df, hue= 'gender')
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(10,4),dpi=150)
sns.countplot(x='residencyId',data=df, hue= 'gender')
plt.xticks(rotation=90)

In [ ]:
df.info()

In [ ]:
num_df = df.select_dtypes(include=['int64','float64']).columns
print(num_df)

In [ ]:
sns.countplot(data=num_df)

In [ ]:
cat_df = df.select_dtypes(include=['object']).columns
print(cat_df)

In [ ]:
sns.countplot(cat_df)

In [ ]:
df_dummies = pd.get_dummies(df, drop_first=True)

In [ ]:
df_dummies.head(10)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
df_dummies_std = sc.fit_transform(df_dummies)
#df_dummies_std is now a numpy array instead of a Pandas Dataframe. Need to convert it back
new_df = pd.DataFrame(sc.fit_transform(df_dummies), columns=df_dummies.columns, index=df_dummies.index)

In [ ]:
new_df.head()

In [ ]:
new_df.columns[1]

In [ ]:
X = new_df.drop( axis=1)
y = new_df[7]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)